# Worksheet 08

Name:  Aryan Agarwal  
UID: U36516144

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [2]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[4.616596830583689, 4.243162738797952, 5.3400462562431015, 7.5145877180636145, 4.421943624818705, 6.417117246227365, 5.463607852543691, 4.583552736600412, 4.778409469068305, 5.206609911844623]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [3]:
mean = 8
stdev = 1

c2 = np.random.normal(mean , stdev , 10 ).tolist()
print(c2)

[6.69023156520095, 7.5543164543987675, 8.296783985247293, 7.032863944502537, 7.488243945387643, 9.112439085373625, 8.534201613146184, 9.960387653913434, 7.885278372588203, 5.202988860816953]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [4]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[5.202988860816953, 7.885278372588203, 5.206609911844623, 9.960387653913434, 4.778409469068305, 4.583552736600412, 5.463607852543691, 6.417117246227365, 8.534201613146184, 4.421943624818705]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

c1 has mean 5 and stdev 1. c2 has mean 8 and stdev 1

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [5]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[5.202988860816953, 5.206609911844623, 4.778409469068305, 4.583552736600412, 5.463607852543691, 6.417117246227365, 4.421943624818705]
[7.885278372588203, 9.960387653913434, 8.534201613146184]
P(C_1) = 0.7,  P(C_2) = 0.3
mean_1 = 5.153461385988579,  mean_2 = 8.79328921321594
var_1 = 0.3855532491908564,  var_2 = 0.7512429471619725


theyre close

e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [6]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x.append( pdf_i[1] * prob_c[1] / prob_x )
    
probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  5.202988860816953
probability of observing that point if it came from cluster 0 =  1.0262246481208213
probability of observing that point if it came from cluster 1 =  5.82684668228866e-06
point =  7.885278372588203
probability of observing that point if it came from cluster 0 =  1.2979630872911604e-11
probability of observing that point if it came from cluster 1 =  0.25579891427610507
point =  5.206609911844623
probability of observing that point if it came from cluster 0 =  1.024942093787123
probability of observing that point if it came from cluster 1 =  5.962561849289863e-06
point =  9.960387653913434
probability of observing that point if it came from cluster 0 =  1.8247123154212912e-34
probability of observing that point if it came from cluster 1 =  0.15886803171859218
point =  4.778409469068305
probability of observing that point if it came from cluster 0 =  0.6446830692254728
probability of observing that point if it came from cluster 1 =  3.334511687752693e-07
point = 

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [7]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]
var = [sum([prob_c0_x[i] * (data[i] - mean[0])**2 for i in range(len(data))]) / sum(prob_c0_x), sum([prob_c1_x[i] * (data[i] - mean[1])**2 for i in range(len(data))]) / sum(prob_c1_x)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.6758674625414367,  P(C_2) = 0.3241325374585632
mean_1 = 5.108344571357386,  mean_2 = 8.6163699356569
var_1 = 0.340272885614363,  var_2 = 1.0844246941299014


they arent very different

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [8]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)


for p in data:
    pdf_i = [norm.pdf(p, mean[j], var[j]) for j in range(k)]
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x.append( pdf_i[1] * prob_c[1] / prob_x )
    
probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


5.202988860816953
Probability of coming from C_1 = 0.9988974715854985
Probability of coming from C_2 = 0.0011025284145014314

7.885278372588203
Probability of coming from C_1 = 2.8782563087258616e-14
Probability of coming from C_2 = 0.9999999999999712

5.206609911844623
Probability of coming from C_1 = 0.9988824794303655
Probability of coming from C_2 = 0.0011175205696346473

9.960387653913434
Probability of coming from C_1 = 1.0097469691864653e-43
Probability of coming from C_2 = 1.0

4.778409469068305
Probability of coming from C_1 = 0.9995413083669993
Probability of coming from C_2 = 0.0004586916330006551

4.583552736600412
Probability of coming from C_1 = 0.9995094755077322
Probability of coming from C_2 = 0.0004905244922678596

5.463607852543691
Probability of coming from C_1 = 0.9962232656990122
Probability of coming from C_2 = 0.0037767343009878457

6.417117246227365
Probability of coming from C_1 = 0.03087548083398988
Probability of coming from C_2 = 0.9691245191660101

8.53420

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [9]:
label = []

for i in range(len(data)):
    if prob_c0_x[i] < prob_c1_x[i]:
        label.append(1)
    else:
        label.append(0)

print(label)

[0, 1, 0, 1, 0, 0, 0, 1, 1, 0]


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |disagree|
| A  C |disagree|
| A  D |disagree|
| A  E |disagree|
| B  C |agree|
| B  D |disagree|
| B  E |disagree|
| C  D |disagree|
| C  E |disagree|
| D  E |disagree|


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

N!/(2!(N-2)!)

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

36

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

11 agreements, 25 disagree

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.